In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload best_lstm_model.pt here


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [ ]:
# Simulated input to mimic real test data setup
# You can replace this with actual feature values later

# Simulate features like svr2, svr3, svr4
np.random.seed(42)
data_size = 500
df_features = pd.DataFrame({
    'svr2': np.random.normal(48, 11, data_size),
    'svr3': np.random.normal(52, 9, data_size),
    'svr4': np.random.normal(49, 10, data_size),
    'svr1': np.random.normal(50, 10, data_size)  # target
})

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(df_features[['svr2', 'svr3', 'svr4']])
y_scaled = scaler_y.fit_transform(df_features[['svr1']])

def create_dataset(X, y, lookback=50):
    Xs, ys = [], []
    for i in range(len(X) - lookback):
        Xs.append(X[i:i+lookback])
        ys.append(y[i+lookback])
    return np.array(Xs), np.array(ys)

lookback = 50
X_seq, y_seq = create_dataset(X_scaled, y_scaled, lookback)
X_test = X_seq[-200:]
y_test = y_seq[-200:]


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        return self.fc(out[:, -1, :])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size=3, hidden_size=64, num_layers=1).to(device)
model.load_state_dict(torch.load("best_lstm_model.pt", map_location=device))
model.eval()
print("✅ Model loaded successfully.")


In [ ]:
test_loader = DataLoader(TensorDataset(torch.tensor(X_test, dtype=torch.float32)), batch_size=64)
y_pred_batches = []

with torch.no_grad():
    for xb in test_loader:
        xb = xb[0].to(device)
        preds = model(xb).cpu().numpy()
        y_pred_batches.append(preds)

y_pred = np.vstack(y_pred_batches)
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test)

rmse = mean_squared_error(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)

print(f"✅ RMSE: {rmse:.2f}, MAE: {mae:.2f}")


In [ ]:
df_output = pd.DataFrame({
    'actual': y_test_inv.flatten(),
    'predicted': y_pred_inv.flatten()
})
df_output.to_csv("lstm_predictions_from_pt.csv", index=False)
print("📁 Saved as lstm_predictions_from_pt.csv")


In [ ]:
from google.colab import files
files.download("lstm_predictions_from_pt.csv")
